In [52]:
%run basics_sensorSignal.py
%matplotlib

Using matplotlib backend: MacOSX


#### TAN = c(NH$_3$ + c(NH$_4^+$)

sensor response is sigmoidal. It mainly depends on the response time 

In [ ]:
c_nh3, step = 2.5, 0.25
t = np.linspace(0, 60, num=int(100/step+1))

par_sens1 = dict({'max conc': c_nh3, 'response time': 10., 'background signal': 5E-6})
par_sens2 = dict({'max conc': c_nh3, 'response time': 50., 'background signal': 5E-6})
# ...........................................
dfSig1, t1_10, t1_90 = _sensor_response(t=t, par_sens=par_sens1, y=0, function='gompertz',
                                       direction='increase')
dfSig2, t2_10, t2_90 = _sensor_response(t=t, par_sens=par_sens2, y=0, function='gompertz',
                                       direction='increase')
dfSig3, t3_10, t3_90 = _sensor_response(t=t, par_sens=par_sens1, y=0, function='gompertz',
                                       direction='decline')
dfSig4, t4_10, t4_90 = _sensor_response(t=t, par_sens=par_sens2, y=0, function='gompertz',
                                       direction='decline')


# .........................................................................
# plot sensor response
fig, ax = plt.subplots(figsize=(7,4))
ax.set_title('Sensor response', loc='left')

# sensor 1
ax.plot(t, dfSig1, 'k', label='fast response')
ax.axhline(dfSig1['signal'].max()*0.1, lw=0.5, ls=':', color='darkorange'), 
ax.axhline(dfSig1['signal'].max()*0.9, lw=0.5, ls=':', color='darkorange')

# sensor 2
ax.plot(t, dfSig2, ':k', label='slow response')
ax.axhline(dfSig2['signal'].max()*0.1, lw=0.5, ls=':', color='crimson'), 
ax.axhline(dfSig2['signal'].max()*0.9, lw=0.5, ls=':', color='crimson')

# sensor 3
ax.plot(t, dfSig3, 'slategrey', label='fast response - decline')

# sensor 4
ax.plot(t, dfSig4, color='slategrey', ls=':', label='slow response - decline')

ax.set_xlabel('Time / s'), ax.set_ylabel('Signal / V')
plt.legend(loc='lower right', bbox_to_anchor=(1., 0.35))
plt.xlim(-5, t[-1]*1.05)
sns.despine()

sensor signal depends on concentration as well

In [ ]:
start_time, end_time = 0, 60
nsample, nper = 10*(end_time-start_time), 3
dfconc1, D1, P1, t1 = _target_concentration(c_nh3=c_nh3, tstart=start_time, tstop=end_time)
dfconc2, D2, P2, t2 = _target_concentration(N=nsample, nP=nper, c_nh3=c_nh3, 
                                            tstart=start_time, tstop=end_time)

# .................................
fs = 10
fig1, ax1 = plt.subplots(figsize=(7,3))
ax1.set_xlabel('Time / s', fontsize=fs)
ax1.set_ylabel('Analyte concentration', fontsize=fs)
ax1.plot(dfconc1, color='k', lw=1., marker='.', fillstyle='none', 
         label='N:{:.0f}, period: {:.0f}s, freq: 1'.format(end_time-start_time, P1))
ax1.plot(dfconc2, color='darkorange', marker='.', fillstyle='none', lw=0.75,
         label='N:{:.0f}, period: {:.0f}s, freq: {:.0f}'.format(nsample, P1, nper))
ax1.legend(loc=0, fontsize=fs*0.7) #, ax1.set_ylim(-0.05, 1.6)
plt.tight_layout(), sns.despine()
plt.show()

time to combine

signal ~ concentration(t) * response time * some other factors

In [60]:
# gloabl parameter
c_nh3 = 2.5                          # maximal ammonia concentration 
# concentration fluctuation
tstart, tend, step = 0, 60*3, 0.05   # sec; time range of fluctuation
nP = 3.5                             # number of cycles per period
# sensor settings
tstart_sens = 0                      # sec; start time range of sensor response            
t_resp = 12.5                          # sensor response time
t_sample = 10.                       # sampling rate of the sensor; sec

# ........................................................................................
# target concentration
dfconc, D, P, trange = _target_concentration(nP=nP, c_nh3=c_nh3, tstart=tstart, tstop=tend)
print('puls width: ', D)

# .............................................
# sensor time range
sens_time0 = np.linspace(0, D, num=int((D - tstart_sens) / step + 1))

# .............................................
# plotting 
par_meas = dict({'start':tstart, 'stop':tend})
par_sens = dict({'max conc': c_nh3, 'response time': t_resp, 'background signal': 5E-5})
arg_fig = dict({'figsize': (6,3), 'fontsize': 10,
                'title': 'sensor response to fluctuating analyte concentration'})

puls width:  25.714285714285715


In [61]:
fig, ax, ddata = plot_sensorresponse(dfconc=dfconc, sens_time0=sens_time0, step=step, D=D, 
                                     tstart_sens=tstart_sens, par_meas=par_meas, 
                                     par_sens=par_sens, arg_fig=arg_fig, plotCheck=False)